# TASK 4

Создайте соединение с базой данных с помощью библиотеки sqlite3.

In [1]:
import pandas as pd
import sqlite3 as sq
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
conn = sq.connect('/content/drive/MyDrive/Google_Drive/School21/Day11/checking-logs.sqlite')

Создайте соединение с базой данных с помощью библиотеки sqlite3.
Получите схему таблицы test.
Получите только первые 10 строк таблицы test, чтобы проверить, как она выглядит.

In [30]:
pd.io.sql.read_sql('PRAGMA table_info(test);', conn)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


In [31]:
pd.io.sql.read_sql('SELECT * FROM test LIMIT 10', conn)

,index,uid,labname,first_commit_ts,first_view_ts
0,3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


In [32]:
pd.io.sql.read_sql('PRAGMA table_info(deadlines)', conn)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,labs,TEXT,0,None,0
2,2,deadlines,INTEGER,0,None,0


Найдите среди всех пользователей минимальное значение этой самой дельты (периода времени от даты первого коммита пользователя до срока сдачи соответствующего лабораторного задания), используя только один запрос.
Для этого сджойните данные своей таблицы с данными таблицы deadlines
Период времени должен отображаться в часах.
Не учитывайте лабораторное задание project1 с более длительным сроком сдачи, поскольку оно будет выделяться на фоне других.
Сохраните значение в датафрейме df_min с соответствующим uid.

In [33]:
df_min = pd.read_sql("""SELECT test.uid, min((strftime('%s', first_commit_ts) - deadlines)/60/60) as min  
                        FROM test JOIN deadlines on test.labname = deadlines.labs
                        WHERE labname not like 'project1'""", conn)

In [34]:
df_min.to_sql('df_min',  con = conn, if_exists='replace')

Выполните те же самые операции для максимального значения дельты, используя только один запрос. Название итогового датафрейма — df_max.

In [35]:
df_max = pd.read_sql("""SELECT test.uid, max((strftime('%s', first_commit_ts) - deadlines)/60/60) as max  
                        FROM test JOIN deadlines on test.labname = deadlines.labs
                        WHERE labname not like 'project1'""", conn)
df_max

,uid,max
0,user_25,-2


In [36]:
df_max.to_sql('df_max',  con = conn, if_exists='replace')

Выполните те же самые операции для среднего значения дельты, используя только один запрос. На этот раз ваш итоговый датафрейм не должен включать столбец uid; имя датафрейма — df_avg.

In [37]:
df_avg = pd.read_sql("""SELECT avg((strftime('%s', first_commit_ts) - deadlines)/60/60) as avg  
                        FROM test JOIN deadlines on test.labname = deadlines.labs
                        WHERE labname not like 'project1'""", conn)
df_avg

,avg
0,-89.125


In [38]:
df_avg.to_sql('df_avg',  con = conn, if_exists='replace')

Мы хотим проверить предположение о том, что у пользователей, посетивших ленту новостей всего несколько раз, меньше период времени между датой первого коммита и сроком сдачи лабораторного задания. Для этого вам необходимо рассчитать коэффициент корреляции между количеством просмотров страниц и разницей между датами.
Используя только один запрос, создайте таблицу со столбцами: uid, avg_diff, pageviews.
uid — это uid, существующие в таблице test.
avg_diff — среднее значение дельты (периода времени между датой первого коммита и сроком сдачи лабораторного задания) для каждого пользователя.
pageviews — количество посещений ленты новостей одним пользователем.
Не учитывайте лабораторное задание project1.
Сохраните результаты в датафрейме views_diff.
Используйте метод corr() библиотеки Pandas для вычисления коэффициента корреляции между количеством просмотров и значением дельты.

In [39]:
views_diff = pd.read_sql("""SELECT test.uid, (strftime('%s', first_commit_ts) - deadlines)/60/60 as avg_diff, pageviews
                        FROM test 
                        JOIN deadlines on test.labname = deadlines.labs
                        JOIN (SELECT pageviews.uid, count() as pageviews
                        FROM pageviews
                        GROUP BY uid) as page
                        on page.uid = test.uid
                        WHERE labname not like 'project1'
                        GROUP BY test.uid""", conn)
views_diff

,uid,avg_diff,pageviews
0,user_1,-107,28
1,user_10,-112,89
2,user_14,-84,143
3,user_17,-78,47
4,user_18,-10,3
5,user_19,-82,16
6,user_21,-124,10
7,user_25,-105,179
8,user_28,-99,149
9,user_3,-135,317


In [40]:
views_diff.corr()

,avg_diff,pageviews
avg_diff,1.000000,-0.380881
pageviews,-0.380881,1.000000


Закройте соединение с базой данных.

In [41]:
conn.close()